<a href="https://colab.research.google.com/github/joaovictor42/arquitetura-de-computadores/blob/main/Exercicio_APIs%2C_Crawlers%2C_Scrapers_e_LGDP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


## APIs, Crawlers, Scrapers e LGDP  
**Disciplina:** Banco de Dados  
**Professor:** Diego Pessoa  
**Grupo:** João Victor Alcoforado de Araujo

### **1.1. Qual é a diferença entre APIs, Scrapers e Crawlers e em que situações cada um é usado?**

Application Programming Interface (API) é um padrão de comunicação que permite o acesso e o envio de dados entre aplicações por meio de uma interface estruturada e documentada. APIs são interfaces robustas para a troca de dados, pois expõem métodos que possibilitam a customização de operações, como filtragem, ordenação e projeção de resultados. Além da coleta, APIs também são amplamente utilizadas para o envio de dados. O uso de uma interface bem definida permite a aplicação de restrições como controle de concorrência, gerenciamento de acesso, atualização em tempo real e definição de acordos de nível de serviço (SLAs). Por isso, APIs são comumente empregadas na comunicação entre aplicações, como na integração entre front-end e back-end, e na disponibilização governada de dados.
    
Scrapers são programas que simulam o comportamento humano ao navegar por páginas web, com o objetivo de extrair sistematicamente informações disponíveis no DOM (Document Object Model). A técnica de web scraping refere-se à coleta de dados da web por meio desses scrapers. No entanto, como páginas web geralmente contêm dados não estruturados, a extração de informações específicas pode exigir técnicas avançadas de processamento e manipulação de dados. Além disso, a estrutura das páginas pode mudar sem aviso prévio, não havendo garantias de estabilidade ou SLA. Scrapers são comumente utilizados quando não há APIs disponíveis para acessar os dados desejados ou quando é necessário capturar informações renderizadas diretamente no conteúdo visual da página.
    
Crawlers, por sua vez, são programas similares aos scrapers, mas com propósito mais amplo. Eles acessam um grande volume de páginas web de forma automatizada, seguindo links encontrados nas páginas para descobrir e visitar novos conteúdos. Os crawlers operam de maneira genérica, coletando informações de diversas páginas em larga escala. São amplamente utilizados por mecanismos de busca, como o Google, para indexar e mapear a web.


### **1.2. Quais são três exemplos de PII (Personally Identifiable Information) e quais técnicas podem ser aplicadas para anonimizá-los de acordo com a LGPD (Lei Geral de Proteção de Dados)?**

Exemplos de informações pessoalmente identificáveis (PII) incluem: nome, Cadastro de Pessoa Física (CPF) e número de telefone. Essas informações, por permitirem a identificação direta ou indireta de um indivíduo, são consideradas PII.

As principais técnicas de anonimização recomendadas pela LGPD são: supressão, encobrimento, mascaramento e generalização. A supressão consiste na remoção total de dados identificáveis, como eliminar o campo de CPF ou nome de um conjunto de dados. O encobrimento envolve ocultar parte da informação, como exibir apenas os últimos dígitos de um CPF. O mascaramento consiste em substituir os dados reais por valores fictícios ou irreversíveis, como a aplicação de um hash, de modo que o dado original não possa ser recuperado. Por fim, a generalização envolve substituir informações por categorias mais amplas, como a substituição da renda exata por uma faixa de renda.

### **2. Crie um script para consumir dados da API do IBGE para listar os nomes das cidades de um estado específico (2,0)**

In [40]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re

def obter_municipios(uf_estado: str) -> list[str]:
    # Endpoint do IBGE - distritos por estado
    url = f"https://servicodados.ibge.gov.br/api/v1/localidades/estados/{uf_estado.upper()}/distritos"
    response = requests.get(url)
    response.raise_for_status()
    municipios_data = response.json()
    return [data['nome'] for data in municipios_data]

uf_estado = 'PB'
municipios: list[str] = obter_municipios(uf_estado)
for municipio in municipios[:15]:
    print(municipio)

Água Branca
Aguiar
Alagoa Grande
Alagoa Nova
Alagoinha
Alcantil
Algodão de Jandaíra
Alhandra
Amparo
Aparecida
Araçagi
Arara
Araruna
Areia
Cepilho


### **3. Crie um Scraper para ler os dados dos docentes da página do PPGTI (https://www.ifpb.edu.br/ppgti/programa/corpo-docente) - construa um dataframe que liste o nome, linha de pesquisa, url do lattes e e-mail de cada professor (5,0)**


In [39]:
url = "https://www.ifpb.edu.br/ppgti/programa/corpo-docente"
soup = BeautifulSoup(requests.get(url).content, "html.parser")

regex_linha = regex_linha = re.compile(r"Linha de Pesquisa:\s*(.+?)(?=\s*Currículo)")
regex_email = re.compile(r"[\w\.-]+@[\w\.-]+")

docentes = []
for h4 in soup.select("h4")[1:]:
    nome = h4.get_text(strip=True) or None
    linha = email = lattes = None

    tag = h4.find_next_sibling()
    while tag and tag.name != "h4":
        if tag.name == "p":
            text = tag.get_text(strip=True)
            if linha_match := regex_linha.search(text):
                linha = linha_match.group(1)
            if email_match := regex_email.search(text):
                email = email_match.group(0)
            if link := tag.find("a", href=True):
                lattes = link["href"]
        tag = tag.find_next_sibling()

    docentes.append({"Nome": nome, "Linha de Pesquisa": linha, "Lattes": lattes, "E-mail": email})

df = pd.DataFrame(docentes)
df = df.dropna(how="all")
df

,Nome,Linha de Pesquisa,Lattes,E-mail
0,Dr. Alex Sandro da Cunha Rêgo,Ciência de Dados e Inteligência Artificial,https://lattes.cnpq.br/1582109846489096,alex@ifpb.edu.br
1,Dr. Cleumar da Silva Moreira,Redes e Sistemas Distribuídos,http://lattes.cnpq.br/5183105830068378,cleumar.moreira@ifpb.edu.br
2,Dra. Damires Yluska de Souza Fernandes,Ciência de Dados e Inteligência Artificial,http://lattes.cnpq.br/9411293540915384,damires@ifpb.edu.br
3,Dr. Danyllo Wagner Albuquerque,Gestão e Desenvolvimento de Sistemas,mailto:diego.pessoa@ifpb.edu.br,danyllo.albuquerque@ifpb.edu.br
4,Dr.Diego Ernesto Rosa Pessoa,Ciência de Dados e Inteligência Artificial,http://lattes.cnpq.br/4442363726982518,diego.pessoa@ifpb.edu.br
5,Dr. Francisco Dantas Nobre Neto,Ciência de Dados e Inteligência Artificial,http://lattes.cnpq.br/7562809854970378,dantas.nobre@ifpb.edu.br
6,Dr. Francisco Petrônio Alencar de Medeiros,Gestão e Desenvolvimento de Sistemas,http://lattes.cnpq.br/9716270626654261,petronio@ifpb.edu.br
7,Dra. Heremita Brasileiro Lira,Gestão e Desenvolvimento de Sistemas,http://lattes.cnpq.br/3532701873325164,heremita@ifpb.edu.br
8,Dra. Juliana Dantas Ribeiro Viana de Medeiros,Gestão e Desenvolvimento de Sistemas,http://lattes.cnpq.br/9730254173461923,juliana.medeiros@ifpb.edu.br
9,Dr. Katyusco de Farias Santos,Gestão e Desenvolvimento de Sistemas,http://lattes.cnpq.br/1246085373474860,katyusco.santos@ifpb.edu.br
